In [1]:
import monai

In [3]:
# Make a copick project
import os
import shutil

config_blob = """{
    "name": "czii_cryoet_mlchallenge_2024",
    "description": "2024 CZII CryoET ML Challenge training data.",
    "version": "1.0.0",

    "pickable_objects": [
        {
            "name": "apo-ferritin",
            "is_particle": true,
            "pdb_id": "4V1W",
            "label": 1,
            "color": [  0, 117, 220, 128],
            "radius": 60,
            "map_threshold": 0.0418
        },
        {
          "name" : "beta-amylase",
            "is_particle": true,
            "pdb_id": "8ZRZ",
            "label": 2,
            "color": [255, 255, 255, 128],
            "radius": 90,
            "map_threshold": 0.0578  
        },
        {
            "name": "beta-galactosidase",
            "is_particle": true,
            "pdb_id": "6X1Q",
            "label": 3,
            "color": [ 76,   0,  92, 128],
            "radius": 90,
            "map_threshold": 0.0578
        },
        {
            "name": "ribosome",
            "is_particle": true,
            "pdb_id": "6EK0",
            "label": 4,
            "color": [  0,  92,  49, 128],
            "radius": 150,
            "map_threshold": 0.0374
        },
        {
            "name": "thyroglobulin",
            "is_particle": true,
            "pdb_id": "6SCJ",
            "label": 5,
            "color": [ 43, 206,  72, 128],
            "radius": 130,
            "map_threshold": 0.0278
        },
        {
            "name": "virus-like-particle",
            "is_particle": true,
            "label": 6,
            "color": [255, 204, 153, 128],
            "radius": 135,
            "map_threshold": 0.201
        },
        {
            "name": "membrane",
            "is_particle": false,
            "label": 8,
            "color": [100, 100, 100, 128]
        },
        {
            "name": "background",
            "is_particle": false,
            "label": 9,
            "color": [10, 150, 200, 128]
        }
    ],

    "overlay_root": "./kaggle/working/overlay",

    "overlay_fs_args": {
        "auto_mkdir": true
    },

    "static_root": "./kaggle/input/czii-cryo-et-object-identification/train/static"
}"""

copick_config_path = "./kaggle/working/copick.config"
output_overlay = "./kaggle/working/overlay"


with open(copick_config_path, "w") as f:
    f.write(config_blob)
    
# Update the overlay
# Define source and destination directories
source_dir = './kaggle/input/czii-cryo-et-object-identification/train/overlay'
destination_dir = './kaggle/working/overlay'

# Walk through the source directory
for root, dirs, files in os.walk(source_dir):
    # Create corresponding subdirectories in the destination
    relative_path = os.path.relpath(root, source_dir)
    target_dir = os.path.join(destination_dir, relative_path)
    os.makedirs(target_dir, exist_ok=True)
    
    # Copy and rename each file
    for file in files:
        if file.startswith("curation_0_"):
            new_filename = file
        else:
            new_filename = f"curation_0_{file}"
            
        
        # Define full paths for the source and destination files
        source_file = os.path.join(root, file)
        destination_file = os.path.join(target_dir, new_filename)
        
        # Copy the file with the new name
        shutil.copy2(source_file, destination_file)
        print(f"Copied {source_file} to {destination_file}")

Copied ./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/ribosome.json to ./kaggle/working/overlay/ExperimentRuns/TS_5_4/Picks/curation_0_ribosome.json
Copied ./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/virus-like-particle.json to ./kaggle/working/overlay/ExperimentRuns/TS_5_4/Picks/curation_0_virus-like-particle.json
Copied ./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/beta-galactosidase.json to ./kaggle/working/overlay/ExperimentRuns/TS_5_4/Picks/curation_0_beta-galactosidase.json
Copied ./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/beta-amylase.json to ./kaggle/working/overlay/ExperimentRuns/TS_5_4/Picks/curation_0_beta-amylase.json
Copied ./kaggle/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns/TS_5_4/Picks/apo-ferritin.json to ./kaggle/working/overlay/ExperimentRuns/TS_5_4/Picks/c

In [5]:
import copick
import numpy as np
from tqdm import tqdm

root = copick.from_file(copick_config_path)

copick_user_name = "copickUtils"
copick_segmentation_name = "paintedPicks"
voxel_size = 10

/var/folders/l1/vt4hj2ln3hs6c97_qllsy7lh0000gn/T/ipykernel_17624/1521813977.py:5: DeprecationWarning: config_type not found in config file, defaulting to filesystem
  root = copick.from_file(copick_config_path)


In [ ]:
# import os
# import numpy as np
# from tqdm import tqdm
# from pathlib import Path
# from PIL import Image  # For saving PNGs

# # Define tomogram types
# tomo_tpye_list = ["ctfdeconvolved", "denoised", "isonetcorrected", "wbp"]

# # Configuration for directories
# train_label_dir = Path('./datasets/labels/train')
# train_image_dir = Path('./datasets/images/train')
# val_label_dir = Path('./datasets/labels/val')
# val_image_dir = Path('./datasets/images/val')
# metadata_dir = Path('./datasets/metadata')
# os.makedirs(metadata_dir, exist_ok=True)

# for dir_path in [train_label_dir, train_image_dir, val_label_dir, val_image_dir]:
#     dir_path.mkdir(parents=True, exist_ok=True)

# # Parameters
# slice_size = (224, 224)  # Patch size
# stride = 112             # Stride for slicing

# def slice_volume(volume, slice_size, stride):
#     """
#     Slices a 3D volume into 2D patches with the specified size and stride.
#     Args:
#         volume (np.ndarray): 3D array to slice (D, H, W).
#         slice_size (tuple): Size of the 2D slices (height, width).
#         stride (int): Stride for slicing.
#     Returns:
#         slices (list): List of slices with metadata [(z, y, x, patch)].
#     """
#     D, H, W = volume.shape
#     patch_h, patch_w = slice_size
#     slices = []

#     for z in range(D):  # Process each slice independently
#         for y in range(0, H - patch_h + 1, stride):
#             for x in range(0, W - patch_w + 1, stride):
#                 patch = volume[z, y:y + patch_h, x:x + patch_w]
#                 slices.append((z, y, x, patch))
#     return slices

# def save_as_png(volume, save_path):
#     """
#     Save a 2D array as a PNG file.
#     Args:
#         volume (np.ndarray): 2D array to save.
#         save_path (Path): Path to save the PNG file.
#     """
#     volume = ((volume - volume.min()) / (volume.max() - volume.min()) * 255).astype(np.uint8)
#     img = Image.fromarray(volume)
#     img.save(save_path)
    
# def save_as_png_without_scaling(volume, save_path):
#     """
#     Save a 2D array as a PNG file without scaling the values.
#     Args:
#         volume (np.ndarray): 2D array to save.
#         save_path (Path): Path to save the PNG file.
#     """
#     # Ensure the volume is in int32 format
#     volume = volume.astype(np.int32)
    
#     # Debug: Check unique values before saving
#     print(f"Unique values before saving: {np.unique(volume)}")

#     # Save as mode="I" to preserve integer values
#     img = Image.fromarray(volume, mode="I")
#     img.save(save_path)

# def process_and_save(data_dict, image_dir, label_dir, metadata_dir, tomo_type, vol_idx):
#     """
#     Processes and saves sliced images and labels as PNGs.
#     Args:
#         data_dict (dict): Dictionary containing 'image' and 'label'.
#         image_dir (Path): Directory to save image slices.
#         label_dir (Path): Directory to save label slices.
#         metadata_dir (Path): Directory to save metadata.
#         tomo_type (str): Tomogram type (for naming).
#         vol_idx (int): Volume index (for naming).
#     """
#     image_volume = data_dict["image"]
#     label_volume = data_dict["label"]

#     # Save original shape metadata
#     metadata_file = metadata_dir / f"{tomo_type}_vol_{vol_idx:03d}_metadata.txt"
#     with open(metadata_file, "w") as f:
#         f.write(f"Original shape: {image_volume.shape}\n")

#     # Slice and save images and labels
#     image_slices = slice_volume(image_volume, slice_size, stride)
#     label_slices = slice_volume(label_volume, slice_size, stride)
    
#     for (z_img, y_img, x_img, img_patch), (z_lbl, y_lbl, x_lbl, lbl_patch) in tqdm(
#         zip(image_slices, label_slices), desc=f"Processing {tomo_type} volume {vol_idx}"
#     ):
#         # Save image slice
#         image_filename = f"{tomo_type}_vol_{vol_idx:03d}_z{z_img}_y{y_img}_x{x_img}.png"
#         image_save_path = image_dir / image_filename
#         save_as_png(img_patch, image_save_path)

#         # Save label slice
#         label_filename = f"{tomo_type}_vol_{vol_idx:03d}_z{z_lbl}_y{y_lbl}_x{x_lbl}.png"
#         label_save_path = label_dir / label_filename
#         save_as_png_without_scaling(lbl_patch, label_save_path)

# # Iterate over all tomogram types
# for tomo_type in tomo_tpye_list:
#     print(f"Processing \"{tomo_type}\" data...")
#     for vol_idx, run in enumerate(root.runs):
#         # Load image and label data
#         tomogram = run.get_voxel_spacing(voxel_size).get_tomogram(tomo_type).numpy()
#         segmentation = run.get_segmentations(
#             name=copick_segmentation_name,
#             user_id=copick_user_name,
#             voxel_size=voxel_size,
#             is_multilabel=True
#         )[0].numpy()
#         print("Original segmentation unique values:", np.unique(segmentation))

#         data_dict = {"image": tomogram, "label": segmentation}

#         # Determine dataset type (train/val)
#         is_test = (vol_idx == len(root.runs) - 1)
#         label_dir = val_label_dir if is_test else train_label_dir
#         image_dir = val_image_dir if is_test else train_image_dir

#         # Process and save slices
#         process_and_save(data_dict, image_dir, label_dir, metadata_dir, tomo_type, vol_idx)

# print("Processing complete.")

In [4]:
# image_dir = Path(image_dir)
# label_dir = Path(label_dir) if label_dir else None





# # Collect sorted file paths for images
# image_files = sorted(list(image_dir.glob("*.png")))
# image_files[:194]

In [6]:
from pathlib import Path

tomo_type_list = ["ctfdeconvolved", "denoised", "isonetcorrected", "wbp"]
# Define directories for saving numpy arrays
train_image_dir = Path('./datasets/train/images')
train_label_dir = Path('./datasets/train/labels')
val_image_dir = Path('./datasets/val/images')
val_label_dir = Path('./datasets/val/labels')

for dir_path in [train_image_dir, train_label_dir, val_image_dir, val_label_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)

for tomo_type in tomo_type_list:
    print(f"Processing \"{tomo_type}\" data...")
    for vol_idx, run in enumerate(root.runs):
        # Load image and label data
        tomogram = run.get_voxel_spacing(voxel_size).get_tomogram(tomo_type).numpy()
        segmentation = run.get_segmentations(
            name=copick_segmentation_name,
            user_id=copick_user_name,
            voxel_size=voxel_size,
            is_multilabel=True
        )[0].numpy()

        # Format run name
        run_name = run.name.replace("\\", "_").replace("/", "_")

        # Determine if this is the last volume
        is_last_volume = (vol_idx == len(root.runs) - 1)

        # Set directories based on whether it's the last volume
        image_dir = val_image_dir if is_last_volume else train_image_dir
        label_dir = val_label_dir if is_last_volume else train_label_dir

        # Save tomogram and segmentation as numpy arrays
        np.save(image_dir / f"{tomo_type}_{run_name}_image.npy", tomogram)
        np.save(label_dir / f"{tomo_type}_{run_name}_label.npy", segmentation)

print("Processing complete.")

Processing "ctfdeconvolved" data...
Processing "denoised" data...
Processing "isonetcorrected" data...
Processing "wbp" data...
Processing complete.


In [3]:
import numpy as np
import torch
from monai.data import DataLoader, Dataset, CacheDataset, decollate_batch
import os
from monai.transforms import (
    Compose, 
    EnsureChannelFirstd, 
    Orientationd,  
    AsDiscrete,  
    RandFlipd, 
    RandRotate90d, 
    NormalizeIntensityd,
    RandCropByLabelClassesd,
)

In [4]:
TRAIN_IMG_DIR = "./datasets/train/images"
TRAIN_LABEL_DIR = "./datasets/train/labels"
VAL_IMG_DIR = "./datasets/val/images"
VAL_LABEL_DIR = "./datasets/val/labels"

train_list = os.listdir(TRAIN_IMG_DIR)
val_list = os.listdir(VAL_IMG_DIR)
train_files = []
valid_files = []


for name in train_list:
    image = np.load(os.path.join(TRAIN_IMG_DIR, f"{name}"))    
    label = np.load(os.path.join(TRAIN_LABEL_DIR, f"{name.replace("image", "label")}"))

    train_files.append({"image": image, "label": label})    

for name in val_list:
    image = np.load(os.path.join(VAL_IMG_DIR, f"{name}"))
    label = np.load(os.path.join(VAL_LABEL_DIR, f"{name.replace("image", "label")}"))

    valid_files.append({"image": image, "label": label})

In [10]:
# Non-random transforms to be cached
non_random_transforms = Compose([
    EnsureChannelFirstd(keys=["image", "label"], channel_dim="no_channel"),
    NormalizeIntensityd(keys="image"),
    Orientationd(keys=["image", "label"], axcodes="ASR")
])



my_num_samples = 1
train_batch_size = 1


# Random transforms to be applied during training
random_transforms = Compose([
    RandCropByLabelClassesd(
        keys=["image", "label"],
        label_key="label",
        spatial_size=[11, 96, 96],
        num_classes=7,
        num_samples=my_num_samples,
        lazy=True,
    ),
    RandRotate90d(keys=["image", "label"], prob=0.5, spatial_axes=[0, 2]),
    RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),    
])


In [ ]:
raw_train_ds = CacheDataset(data=train_files, transform=non_random_transforms, cache_rate=1.0)


train_ds = Dataset(data=raw_train_ds, transform=random_transforms)


# DataLoader remains the same
train_loader = DataLoader(
    train_ds,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=torch.cuda.is_available()
)


In [ ]:
# 첫 번째 배치를 가져옵니다.
for batch in train_loader:
    # 배치의 입력 데이터와 레이블을 가져옵니다.
    x, y = batch['image'], batch['label']
    
    # 입력 데이터와 레이블의 모양을 출력합니다.
    print(f'입력 데이터의 모양: {x.shape}')
    print(f'레이블의 모양: {y.shape}')
    
    # 첫 번째 배치만 확인하면 되므로 break로 루프를 종료합니다.
    break

python(38708) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(38749) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [11]:

raw_valid_ds = CacheDataset(data=valid_files, transform=non_random_transforms, cache_rate=1.0)
valid_ds = Dataset(data=raw_valid_ds, transform=random_transforms)
valid_batch_size = 1

# DataLoader remains the same
valid_loader = DataLoader(
    valid_ds,
    batch_size=valid_batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=torch.cuda.is_available()
)


Loading dataset: 100%|██████████| 4/4 [00:01<00:00,  2.69it/s]


In [12]:
# 첫 번째 배치를 가져옵니다.
for batch in valid_loader:
    # 배치의 입력 데이터와 레이블을 가져옵니다.
    x, y = batch['image'], batch['label']
    
    # 입력 데이터와 레이블의 모양을 출력합니다.
    print(f'입력 데이터의 모양: {x.shape}')
    print(f'레이블의 모양: {y.shape}')
    
    # 첫 번째 배치만 확인하면 되므로 break로 루프를 종료합니다.
    break

입력 데이터의 모양: torch.Size([1, 1, 11, 96, 96])
레이블의 모양: torch.Size([1, 1, 11, 96, 96])


In [ ]:
# import torch
# from PIL import Image
# import numpy as np
# from pathlib import Path
# from torch.utils.data import Dataset, DataLoader
# from collections import defaultdict


# import torch
# from torch.utils.data import Dataset
# import numpy as np
# from pathlib import Path


# class NumpyCryoETDataset(Dataset):
#     def __init__(self, data_dir, num_channels=11, slice_size=(224, 224), stride=112, transform=None):
#         self.data_dir = Path(data_dir)
#         self.num_channels = num_channels
#         self.slice_size = slice_size
#         self.stride = stride
#         self.transform = transform

#         # Collect all numpy files
#         self.image_files = sorted(list(self.data_dir.glob("*_image.npy")))
#         self.label_files = sorted(list(self.data_dir.glob("*_label.npy")))
#         assert len(self.image_files) == len(self.label_files), "Mismatch between image and label files!"

#         # Generate indices for individual patches
#         self.data_indices = self._generate_indices()

#     def _generate_indices(self):
#         """Generate indices for individual patches."""
#         indices = []
#         for file_idx, (image_file, label_file) in enumerate(zip(self.image_files, self.label_files)):
#             image = np.load(image_file)
#             label = np.load(label_file)
#             D, H, W = image.shape

#             for z in range(self.num_channels // 2, D - self.num_channels // 2):
#                 for y in range(0, H - self.slice_size[0] + 1, self.stride):
#                     for x in range(0, W - self.slice_size[1] + 1, self.stride):
#                         indices.append((file_idx, z, y, x))
#         return indices

#     def __len__(self):
#         return len(self.data_indices)

#     def __getitem__(self, idx):
#         # Retrieve patch index
#         file_idx, z, y, x = self.data_indices[idx]

#         # Load numpy arrays
#         image = np.load(self.image_files[file_idx])
#         label = np.load(self.label_files[file_idx])

#         # Extract 2.5D patch
#         input_patch = image[z - self.num_channels // 2:z + self.num_channels // 2 + 1, y:y + self.slice_size[0], x:x + self.slice_size[1]]
#         label_patch = label[z, y:y + self.slice_size[0], x:x + self.slice_size[1]]

#         # Apply transformations if provided
#         if self.transform:
#             input_patch, label_patch = self.transform(input_patch, label_patch)

#         # Convert to PyTorch tensors
#         input_patch = torch.tensor(input_patch, dtype=torch.float32)
#         label_patch = torch.tensor(label_patch, dtype=torch.long)

#         return input_patch, label_patch

# # Example usage
# data_dir = "./datasets/numpy"
# dataset = NumpyCryoETDataset(data_dir, num_channels=11, slice_size=(224, 224), stride=112)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

# for inputs, labels in dataloader:
#     print(f"Inputs shape: {inputs.shape}")  # (B, num_channels, H, W)
#     print(f"Labels shape: {labels.shape}")  # (B, H, W)
#     break

Inputs shape: torch.Size([1, 11, 224, 224])
Labels shape: torch.Size([1, 224, 224])


In [8]:
import torch
import torch.nn as nn
from monai.networks.nets import UNet

class UNet2_5D_v2(nn.Module):
    def __init__(self, out_channels=6):
        super().__init__()
        
        # 초기 3D 처리 레이어
        self.init_3d = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=(11, 3, 3), padding=(0, 1, 1)),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True)
        )
        
        # 2D UNet
        self.unet = UNet(
            spatial_dims=2,
            in_channels=64,  # 3D 컨볼루션 출력 채널
            out_channels=out_channels,
            channels=(128, 256, 512, 1024),
            strides=(2, 2, 2, 2),
            num_res_units=2
        )

    def forward(self, x):
        # x shape: (batch, 1, 11, H, W)
        # 3D 처리
        x = x.unsqueeze(1)
        x = self.init_3d(x)  # (batch, 64, 1, H, W)
        x = x.squeeze(2)     # (batch, 64, H, W)
        
        # 2D UNet
        return self.unet(x)

# 테스트 코드

model = UNet2_5D_v2(out_channels=7)
x = torch.randn(2, 11, 224, 224)
output = model(x)
print(f"Output shape: {output.shape}")  # Expected: (8, 6, 256, 256)

c:\ProgramData\anaconda3\envs\ship\Lib\site-packages\monai\networks\nets\unet.py:130: UserWarning: `len(strides) > len(channels) - 1`, the last 1 values of strides will not be used.
  warnings.warn(f"`len(strides) > len(channels) - 1`, the last {delta} values of strides will not be used.")


Output shape: torch.Size([2, 7, 224, 224])


In [ ]:
for inputs, targets in dataloader:
    print(f"Max label value: {targets.max().item()}")
    print(f"Num classes (model output): {output.shape[1]}")
    break

Max label value: 6
Num classes (model output): 7


In [ ]:
import torch
from monai.losses import DiceLoss
from torch import optim

# Loss and Optimizer
criterion = DiceLoss(to_onehot_y=True, softmax=True)  # Dice Loss with softmax
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, targets in dataloader:
        # Ensure targets are long integers
        targets = targets.long()  # 라벨: (B, H, W)

        # Add channel dimension to targets: (B, H, W) -> (B, 1, H, W)
        targets = targets.unsqueeze(1)
        print(f"Modified Targets shape: {targets.shape}")  # (B, 1, H, W)

        # Forward pass
        outputs = model(inputs)  # 모델 출력: (B, 7, H, W)
        print(f"Outputs shape: {outputs.shape}, Targets shape: {targets.shape}")

        # Compute loss
        loss = criterion(outputs, targets)  # Dice Loss
        print(f"Loss: {loss.item()}")

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print epoch loss
    epoch_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

Modified Targets shape: torch.Size([1, 1, 224, 224])
Outputs shape: torch.Size([1, 7, 224, 224]), Targets shape: torch.Size([1, 1, 224, 224])
Loss: 0.9693675637245178
Modified Targets shape: torch.Size([1, 1, 224, 224])
Outputs shape: torch.Size([1, 7, 224, 224]), Targets shape: torch.Size([1, 1, 224, 224])
Loss: 0.9682437181472778
Modified Targets shape: torch.Size([1, 1, 224, 224])
Outputs shape: torch.Size([1, 7, 224, 224]), Targets shape: torch.Size([1, 1, 224, 224])
Loss: 0.9593340754508972
Modified Targets shape: torch.Size([1, 1, 224, 224])
Outputs shape: torch.Size([1, 7, 224, 224]), Targets shape: torch.Size([1, 1, 224, 224])
Loss: 0.9628776907920837
Modified Targets shape: torch.Size([1, 1, 224, 224])
Outputs shape: torch.Size([1, 7, 224, 224]), Targets shape: torch.Size([1, 1, 224, 224])
Loss: 0.957891047000885
Modified Targets shape: torch.Size([1, 1, 224, 224])
Outputs shape: torch.Size([1, 7, 224, 224]), Targets shape: torch.Size([1, 1, 224, 224])
Loss: 0.9588175415992737

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, Dataset
# import numpy as np

# # Define the model (U-Net)
# class DoubleConv(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(DoubleConv, self).__init__()
#         self.double_conv = nn.Sequential(
#             nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True)
#         )

#     def forward(self, x):
#         return self.double_conv(x)


# class UNet(nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(UNet, self).__init__()
#         self.enc1 = DoubleConv(in_channels, 64)
#         self.pool1 = nn.MaxPool2d(2)
#         self.enc2 = DoubleConv(64, 128)
#         self.pool2 = nn.MaxPool2d(2)
#         self.enc3 = DoubleConv(128, 256)
#         self.pool3 = nn.MaxPool2d(2)
#         self.bridge = DoubleConv(256, 512)
#         self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
#         self.dec3 = DoubleConv(512, 256)
#         self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
#         self.dec2 = DoubleConv(256, 128)
#         self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
#         self.dec1 = DoubleConv(128, 64)
#         self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)

#     def forward(self, x):
#         enc1 = self.enc1(x)
#         enc2 = self.enc2(self.pool1(enc1))
#         enc3 = self.enc3(self.pool2(enc2))
#         bridge = self.bridge(self.pool3(enc3))
#         dec3 = self.dec3(torch.cat([self.upconv3(bridge), enc3], dim=1))
#         dec2 = self.dec2(torch.cat([self.upconv2(dec3), enc2], dim=1))
#         dec1 = self.dec1(torch.cat([self.upconv1(dec2), enc1], dim=1))
#         return self.out_conv(dec1)


# # Instantiate model, loss, and optimizer
# model = UNet(in_channels=11, out_channels=2)  # 2 classes for segmentation
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-4)


# # Training Loop
# for epoch in range(5):  # 5 epochs for demonstration
#     model.train()
#     running_loss = 0.0
#     for inputs, targets in dataloader:
#         optimizer.zero_grad()
#         outputs = model(inputs)  # Forward pass
#         loss = criterion(outputs, targets)  # Compute loss
#         loss.backward()  # Backpropagation
#         optimizer.step()  # Update weights
#         running_loss += loss.item()

#     print(f"Epoch [{epoch+1}/5], Loss: {running_loss/len(dataloader):.4f}")